<a href="https://colab.research.google.com/github/gieoon/.Blob-iOS/blob/master/Craftie_Synthetic_Data_CTGAN_(Larger_Dataset___Customized_CSS_Variables).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- https://sdv.dev/CTGAN/
- https://sdv.dev/CTGAN/api/ctgan.html#module-ctgan
- https://sdv.dev/CTGAN/api/ctgan.html
- https://github.com/sdv-dev/CTGAN/blob/master/ctgan/synthesizers/ctgan.py
- https://towardsdatascience.com/synthesis-of-tabular-financial-data-using-generative-algorithms-e90415f889ba


In [2]:
!pip install ctgan

     |████████████████████████████████| 22.3MB 1.6MB/s 
     |████████████████████████████████| 9.5MB 33.8MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5


In [3]:
DATASET_VERSION = 16 # The version of the dataset used in the code (dataset_{VERSION}.txt)
EPOCHS = 200
DATASET_AUGMENT = 1
JOB_ID = "COLAB"

# Hyperparameters
DISCRIMINATOR_STEPS=1
EMBEDDING_DIM=512
GENERATOR_DIM=(512,512)
DISCRIMINATOR_DIM=(512,512)
DISCRIMINATOR_LEARNING_RATE=5e-3
GENERATOR_LEARNING_RATE=5e-3
LOG_FREQUENCY=True

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd 

train_X = pd.read_csv('/content/drive/MyDrive/Craftie/dataset_encoded_100.csv')#,header=None)
train_X = train_X.fillna(-999)
print(len(train_X.columns))

# def shrink(x):
#   return x / MAX_VALUE

# def reverse_shrink(x):
#   return x * MAX_VALUE

# Drop the last column, it is NaN
train_X = train_X.drop(train_X.columns[-1],axis=1)

# Rename columns for descriptions
# mapping = {df.columns[0]: 'Spaciousness',df.columns[1]:'Creativity',df.columns[2]:'BusinessType',df.columns[3]:'ColorTheme'}
# train_X = train_X.rename(columns=mapping)

# train_X = train_X.fillna(0)
# train_X = train_X.replace(1, MAX_VALUE)
# train_X = train_X.replace(-1, MAX_VALUE)

# train_X = train_X.apply(shrink)
# test_X = test_X.apply(shrink)
frames = []
for _ in range(DATASET_AUGMENT):
  frames.append(train_X)
train_X = pd.concat(frames)
train_X.describe()
# train_X.head(1)

8786


,-999,-999.1,-999.2,-999.3,-999.4,-999.5,-999.6,-999.7,-999.8,-999.9,-999.10,-999.11,0,-999.12,-999.13,-999.14,-999.15,-999.16,-999.17,-999.18,-999.19,-999.20,-999.21,-999.22,-999.23,0.1,-999.24,-999.25,-999.26,-999.27,-999.28,-999.29,-999.30,-999.31,-999.32,-999.33,-999.34,-999.35,0.2,-999.36,...,.Testimonials_Wrapper.font-size.1,.Testimonials_Wrapper.font-weight.1,.Testimonials_Wrapper.height.1,.Testimonials_Wrapper.left.1,.Testimonials_Wrapper.line-height.1,.Testimonials_Wrapper.margin.1,.Testimonials_Wrapper.margin.2,.Testimonials_Wrapper.margin.3,.Testimonials_Wrapper.margin.4,.Testimonials_Wrapper.max-height.1,.Testimonials_Wrapper.max-width.1,.Testimonials_Wrapper.min-height.1,.Testimonials_Wrapper.min-width.1,.Testimonials_Wrapper.padding.1,.Testimonials_Wrapper.padding.2,.Testimonials_Wrapper.padding.3,.Testimonials_Wrapper.padding.4,.Testimonials_Wrapper.right.1,.Testimonials_Wrapper.top.1,.Testimonials_Wrapper.transition.1,.Testimonials_Wrapper.transition.2,.Testimonials_Wrapper.transition.3,.Testimonials_Wrapper.transition.4,.Testimonials_Wrapper.transition.5,.Testimonials_Wrapper.width.1,.Testimonials_Wrapper.z-index.1,-999.1288,-999.1289,-999.1290,-999.1291,-999.1292,-999.1293,-999.1294,-999.1295,-999.1296,-999.1297,-999.1298,-999.1299,0.101,0.5.91
count,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,...,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0,99.0
mean,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,...,0.0,-999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,125.0,1.0,0.0,0.0,0.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,0.5
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,...,0.0,-999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,125.0,1.0,0.0,0.0,0.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,0.5
25%,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,...,0.0,-999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,125.0,1.0,0.0,0.0,0.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,0.5
50%,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,...,0.0,-999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,125.0,1.0,0.0,0.0,0.0,0.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,0.5
75%,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,

In [ ]:
# # (Optional) Using a scaler
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()
# train_X = pd.DataFrame(scaler.fit_transform(train_X), columns=train_X.columns)
# train_X.head(1)

In [7]:
# Creating discrete columns by querying for strings
discrete_columns = [col for col, dt in train_X.dtypes.items() if dt == object]
# [dt for col, dt in train_X.dtypes.items()]
len(discrete_columns)

2866

In [ ]:
from ctgan import CTGANSynthesizer

model = CTGANSynthesizer(
    verbose=True,
    discriminator_decay=1e-6,
    discriminator_steps=DISCRIMINATOR_STEPS,
    embedding_dim=EMBEDDING_DIM,
    generator_dim=GENERATOR_DIM,
    discriminator_dim=DISCRIMINATOR_DIM,
    generator_lr=GENERATOR_LEARNING_RATE,
    discriminator_lr=DISCRIMINATOR_LEARNING_RATE,
    log_frequency=LOG_FREQUENCY
)

model.fit(train_X, discrete_columns, epochs = EPOCHS)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass n_components=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/mixture/_base.py:148: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to duplicate points in X.
  random_state=random_state).fit(X).labels_
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:72: FutureWarning: Pass n_components=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/mixture/_base.py:148: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (10). Possibly due to dupl

In [ ]:
samples = model.sample(10)
# samples = samples.apply(reverse_shrink)
samples

In [ ]:
# for i in range(train_X.shape[1]):
#   print("{},{},{},{} => {}".format(train_X.iloc[0][i], train_X.iloc[5][i], train_X.iloc[10][i], train_X.iloc[20][i], samples[i]))

In [ ]:
filename = '/content/drive/MyDrive/Craftie/CTGAN_v{}_{}_{}_{}_ED_{}_GD_{}_DD_{}_GLR_{}_DLR_{}_DS_{}_LF_{}'.format(
    DATASET_VERSION, 
    EPOCHS, 
    DATASET_AUGMENT,
    JOB_ID,
    EMBEDDING_DIM,
    GENERATOR_DIM,
    DISCRIMINATOR_DIM,
    GENERATOR_LEARNING_RATE,
    DISCRIMINATOR_LEARNING_RATE,
    DISCRIMINATOR_STEPS,
    LOG_FREQUENCY
)
filename

In [ ]:
model.save(filename)

In [ ]:
from ctgan import CTGANSynthesizer
loaded_ctgan = CTGANSynthesizer()
loaded_model = loaded_ctgan.load(filename)

In [ ]:
train_X.head(20)

,multipage,topdown,MobileNav1,Nav1,AboutPage1,ContactPage1,ProductPage1,Hero1,MiniCta1,Products1,Testimonials1,Section1,AdditionalImages1,Footer1,ButtonArrow,Hero,Section1.1,Section2,Section3,Products,Testimonials,MiniCta,AdditionalImages,None,-999,-999.1,-999.2,-999.3,-999.4,-999.5,-999.6,-999.7,-999.8,-999.9,-999.10,-999.11,0,None.1,-999.12,-999.13,...,.Testimonials_Wrapper.min-width.1,.Testimonials_Wrapper.min-width.2,.Testimonials_Wrapper.overflow.1,.Testimonials_Wrapper.padding.1,.Testimonials_Wrapper.padding.2,.Testimonials_Wrapper.padding.3,.Testimonials_Wrapper.padding.4,.Testimonials_Wrapper.padding.5,.Testimonials_Wrapper.padding.6,.Testimonials_Wrapper.padding.7,.Testimonials_Wrapper.padding.8,.Testimonials_Wrapper.position.1,.Testimonials_Wrapper.right.1,.Testimonials_Wrapper.right.2,.Testimonials_Wrapper.text-align.1,.Testimonials_Wrapper.top.1,.Testimonials_Wrapper.top.2,.Testimonials_Wrapper.transition.1,.Testimonials_Wrapper.transition.2,.Testimonials_Wrapper.transition.3,.Testimonials_Wrapper.transition.4,.Testimonials_Wrapper.transition.5,.Testimonials_Wrapper.width.1,.Testimonials_Wrapper.width.2,.Testimonials_Wrapper.z-index.1,Solid.89,-999.1288,-999.1289,-999.1290,-999.1291,-999.1292,-999.1293,-999.1294,-999.1295,-999.1296,-999.1297,-999.1298,-999.1299,0.101,0.5.91
0,multipage,topdown,MobileNav1,Nav1,AboutPage1,ContactPage1,ProductPage1,Hero1,MiniCta1,Products1,Testimonials1,Section1,AdditionalImages1,Footer1,ButtonArrow,Hero,Section1,Section2,Section3,Products,Testimonials,MiniCta,AdditionalImages,None,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,None,-999,-999,...,0,USE_DEFAULT,USE_DEFAULT,0,0,0,0,USE_DEFAULT,USE_DEFAULT,USE_DEFAULT,USE_DEFAULT,USE_DEFAULT,0,USE_DEFAULT,USE_DEFAULT,0,USE_DEFAULT,125,1,0,0,0,0,USE_DEFAULT,-999,Solid,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,0.5
1,multipage,topdown,MobileNav1,Nav1,AboutPage1,ContactPage1,ProductPage1,Hero1,MiniCta1,Products1,Testimonials1,Section1,AdditionalImages1,Footer1,ButtonArrow,Hero,Section1,Section2,Section3,Products,Testimonials,MiniCta,AdditionalImages,None,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,None,-999,-999,...,0,USE_DEFAULT,USE_DEFAULT,0,0,0,0,USE_DEFAULT,USE_DEFAULT,USE_DEFAULT,USE_DEFAULT,USE_DEFAULT,0,USE_DEFAULT,USE_DEFAULT,0,USE_DEFAULT,125,1,0,0,0,0,USE_DEFAULT,-999,Solid,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,0.5
2,multipage,topdown,MobileNav1,Nav1,AboutPage1,ContactPage1,ProductPage1,Hero1,MiniCta1,Products1,Testimonials1,Section1,AdditionalImages1,Footer1,ButtonArrow,Hero,Section1,Section2,Section3,Products,Testimonials,MiniCta,AdditionalImages,None,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,None,-999,-999,...,0,USE_DEFAULT,USE_DEFAULT,0,0,0,0,USE_DEFAULT,USE_DEFAULT,USE_DEFAULT,USE_DEFAULT,USE_DEFAULT,0,USE_DEFAULT,USE_DEFAULT,0,USE_DEFAULT,125,1,0,0,0,0,USE_DEFAULT,-999,Solid,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,0.5
3,multipage,topdown,MobileNav1,Nav1,AboutPage1,ContactPage1,ProductPage1,Hero1,MiniCta1,Products1,Testimonials1,Section1,AdditionalImages1,Footer1,ButtonArrow,Hero,Section1,Section2,Section3,Products,Testimonials,MiniCta,AdditionalImages,None,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,None,-999,-999,...,0,USE_DEFAULT,USE_DEFAULT,0,0,0,0,USE_DEFAULT,USE_DEFAULT,USE_DEFAULT,USE_DEFAULT,USE_DEFAULT,0,USE_DEFAULT,USE_DEFAULT,0,USE_DEFAULT,125,1,0,0,0,0,USE_DEFAULT,-999,Solid,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,0.5
4,multipage,topdown,MobileNav1,Nav1,AboutPage1,ContactPage1,ProductPage1,Hero1,MiniCta1,Products1,Testimonials1,Section1,AdditionalImages1,Footer1,ButtonArrow,Hero,Section1,Section2,Section3,Products,Testimonials,MiniCta,AdditionalImages,None,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,0,None,-999,-999,...,0,USE_DEFAULT,USE_DEFAULT,0,0,0,0,USE_DEFAULT,USE_DEFAULT,USE_DEFAULT,USE_DEFAULT,USE_DEFAULT,0,USE_DEFAULT,USE_DEFAULT,0,USE

In [ ]:
loaded_model.sample(10) #, condition_column='1',condition_value='leftright'